### [92 反转链表 II](https://leetcode.cn/problems/reverse-linked-list-ii/description/?envType=study-plan-v2&envId=top-interview-150)  
<font color=#FFCC33>中等</font>

前序题： [反转链表](./Solution92.ipynb)

我按**指针语义 + 不变量**的方式，把这段解法完整拆解一遍。你会看到它其实是一个“**局部头插反转 + 两次接回**”的标准模板。

---

## 一、整体思路（先给结论）

这道题的核心不是“反转链表”，而是：

> **只反转 `[left, right]` 这一段，并且保证前后链表不断裂。**

该解法分三步：

1. **找到待反转区间前一个节点 `p0`**
2. **原地反转 `[left, right]` 这段链表**
3. **把反转后的子链表接回原链表**

---

## 二、第一步：dummy + p0 的意义

```python
p0 = dummy = ListNode(next=head)
for _ in range(left - 1):
    p0 = p0.next
```

### 1️⃣ 为什么要 dummy？

为了统一处理边界情况：

* `left = 1` 时，反转从头结点开始
* 没有 dummy，就会出现“头指针要特殊处理”的分支

dummy 结构如下：

```
dummy -> 1 -> 2 -> 3 -> 4 -> 5
```

### 2️⃣ p0 是谁？

循环结束后：

```
p0 指向：第 left-1 个节点
p0.next 指向：第 left 个节点（反转起点）
```

示例（left=2, right=4）：

```
dummy -> 1 -> 2 -> 3 -> 4 -> 5
          ^
         p0
```

---

## 三、第二步：局部反转的核心循环（最关键）

```python
pre = None
cur = p0.next
for _ in range(right - left + 1):
    nxt = cur.next
    cur.next = pre
    pre = cur
    cur = nxt
```

这是**标准链表反转模板**，但只执行 `right-left+1` 次。

### 🔁 每一轮循环发生了什么？

假设反转 `[2,3,4]`：

#### 初始状态：

```
pre = None
cur = 2
```

#### 第 1 轮：

```
nxt = 3
2.next = None
pre = 2
cur = 3
```

链表局部形态：

```
2 -> None
```

#### 第 2 轮：

```
nxt = 4
3.next = 2
pre = 3
cur = 4
```

```
3 -> 2 -> None
```

#### 第 3 轮：

```
nxt = 5
4.next = 3
pre = 4
cur = 5
```

```
4 -> 3 -> 2 -> None
```

### 🔒 循环结束后的**不变量**

* `pre` → **反转后的子链表头**
* `cur` → **right 后面的第一个节点**
* `p0.next` → **反转前的 left 节点（现在是尾巴）**

---

## 四、第三步：两次“接回”是整个题的精髓

```python
p0.next.next = cur
p0.next = pre
```

### 1️⃣ 第一行：接“尾巴”

```python
p0.next.next = cur
```

* `p0.next`：原来的 `left` 节点
* 反转后它变成了**尾节点**
* 它应该接上 `right+1` 的节点（即 `cur`）

示意：

```
反转后子链表：
4 -> 3 -> 2 -> None

让 2.next = 5
```

---

### 2️⃣ 第二行：接“头部”

```python
p0.next = pre
```

* `pre` 是反转后的头节点（right）
* `p0` 应该指向它

最终结构：

```
dummy -> 1 -> 4 -> 3 -> 2 -> 5
```

---

## 五、为什么顺序不能反？

注意这两行的**顺序非常重要**：

```python
p0.next.next = cur   # 用的是“旧的 p0.next”
p0.next = pre
```

如果反过来：

```python
p0.next = pre
p0.next.next = cur  # ❌ 语义已经变了
```

那 `p0.next` 就不再是原来的 left 节点了，链表会被破坏。

---

## 六、整个算法的逻辑结构总结

可以记成一句话：

> **“p0 找左边界，cur/pre 反转区间，left 当尾巴接 cur，p0 接 pre”**

或者更工程化一点：

| 指针       | 含义               |
| -------- | ---------------- |
| dummy    | 虚拟头，消除 left=1 特判 |
| p0       | left-1 节点        |
| cur      | 当前反转指针           |
| pre      | 已反转部分的头          |
| cur（结束后） | right+1 节点       |

---

## 七、时间 & 空间复杂度

* 时间复杂度：**O(n)**（单次遍历）
* 空间复杂度：**O(1)**（原地反转）

---

## 八、你可以这样“记忆”这段代码

> **“局部反转 = 标准反转 + 两次接回”**

只要你记住：

* 反转区间前要有 `p0`
* 反转完：

  * `p0.next.next = cur`
  * `p0.next = pre`

这道题你就可以**徒手写出来**。

如果你愿意，下一步我可以：


In [8]:
from typing import Optional


class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next


class Solution:
    def reverseBetween(self, head: Optional[ListNode], left: int, right: int) -> Optional[ListNode]:
        p0 = dummy = ListNode(next=head)
        for _ in range(left - 1):
            p0 = p0.next

        pre = None
        cur = p0.next
        for _ in range(right - left + 1):
            nxt = cur.next
            cur.next = pre
            pre = cur
            cur = nxt

        p0.next.next = cur
        p0.next = pre
        return dummy.next